In [2]:
!pip install mechanicalsoup

In [3]:
import mechanicalsoup
import pandas as pd
import sqlite3

In [4]:
browser = mechanicalsoup.StatefulBrowser()
browser.open("https://www.lonelyplanet.com/egypt/attractions")

<Response [200]>

In [5]:
a_tags = browser.page.find_all("a", attrs={"class": "underline-none"})
"""
categorys = [tag.text for tag in a_tags] 
# categorys = ['Sights', 'Restaurants', 'Entertainment', 'Nightlife', 'Shopping', 'Hotels']
"""

categorys_dict = {tag.text: {"url": f'https://www.lonelyplanet.com{tag["href"]}'} for tag in a_tags}

In [6]:
"""
places = browser.page.find_all("li", attrs={"class": "my-3"})
top_palces_name = [tag.article.a.text for tag in places]
top_palces_description = [tag.article.a["href"] for tag in places]
"""
places = browser.page.find_all("li", attrs={"class": "my-3"})
top_palces = {
    place.article.a.text: {
        "url": f'https://www.lonelyplanet.com{place.article.a["href"]}',
        }
    for place in places
}

In [7]:
# li class="flex my-2 text-sm"
browser.open("https://www.lonelyplanet.com/egypt/st-katherine-protectorate/attractions/st-catherine-s-monastery/a/poi-sig/1136752/1295713")  # for opening the links
details = browser.page.find_all("li", attrs={"class": "flex my-2 text-sm"}) # get all places tags

# for det in details:
#   print(det.text)

Location = browser.page.find("div", attrs={"class": "italic"})
print(Location.text.split(" in "))


['Top choice', 'Sinai']


In [8]:
api = []
categorys = {tag.text: {"url": f'https://www.lonelyplanet.com{tag["href"]}'} for tag in a_tags}
for cat in categorys.items():
  cat_name, cat_url = cat[0], cat[1]["url"]
  browser.open(cat_url)  # for opening the links
  places = browser.page.find_all("li", attrs={"class": "my-3"}) # get all places tags

  # store places accourding to it's category on this dict with name and url
  for place in places:
    place_url = f'https://www.lonelyplanet.com{place.article.a["href"]}'
    place_url = place.article.a["href"]
    browser.open_relative(place_url)
    details = browser.page.find_all("li", attrs={"class": "flex my-2 text-sm"})
    Location = browser.page.find("div", attrs={"class": "italic"})
    description = browser.page.find("div", attrs={"class": "lg:w-full styles_article-body__9UptP"})

    # "url": place_url,
    api.append({
        "location_name": place.article.a.text,
        "category": cat_name, 
        "location": Location.text.split(" in ")[-1],
        "description": description.p.text,
        "details": [det.text for det in details]
        })


In [9]:
data = pd.DataFrame(api)
data.info()
# api

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   location_name  120 non-null    object
 1   category       120 non-null    object
 2   location       120 non-null    object
 3   description    120 non-null    object
 4   details        120 non-null    object
dtypes: object(5)
memory usage: 4.8+ KB


In [10]:
# data.to_dict('records')[0]["category"]
test = {}
for idx, place in enumerate(api):
  category = place["category"]

  # drop category and url from data 
  # place_data = {"name": place["name"], "details": place["details"]}

  if category not in test:
      test[category] = [place]
  else:
    test[category].append(place)